<a id='1.3'></a>
## Import Packages

In [1]:
import pandas as pd
import numpy as np
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from stable_baselines3.common.logger import configure
import pandas as pd
from sklearn.preprocessing import StandardScaler #平均值是0, 標準差是1
from finrl.meta.preprocessor.preprocessors import FeatureEngineer
from stable_baselines3 import PPO

<a id='1.4'></a>
## Create Folders

In [2]:
from finrl import config
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
    CHIPS,
)
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

In [3]:
df = pd.read_csv('Input for Stock-Picked Agent predict.csv')
df = df.fillna(0)

In [4]:
features = ['CostOfGoodsSold',
            'EPS',
            'EquityAttributableToOwnersOfParent_x',
            'IncomeAfterTaxes',
            'IncomeFromContinuingOperations',
            'OtherComprehensiveIncome',
            'Revenue',
            'TAX',
            'TotalConsolidatedProfitForThePeriod',	
            'CapitalStock',
            'CapitalStock_per',	
            'CapitalSurplus',
            'CapitalSurplus_per',	
            'CashAndCashEquivalents',	
            'CashAndCashEquivalents_per',	
            'CurrentAssets',
            'CurrentAssets_per',	
            'Equity',
            'EquityAttributableToOwnersOfParent_y',
            'EquityAttributableToOwnersOfParent_per',	
            'Equity_per',
            'NoncurrentAssets',
            'NoncurrentAssets_per',	
            'NoncurrentLiabilities',
            'NoncurrentLiabilities_per',	
            'OrdinaryShare',
            'OrdinaryShare_per',	
            'OtherCurrentLiabilities',	
            'OtherCurrentLiabilities_per',	
            'OtherEquityInterest',
            'OtherEquityInterest_per',	
            'RetainedEarnings',
            'RetainedEarnings_per',	
            'TotalAssets',
            'TotalAssets_per',	
            'CashBalancesBeginningOfPeriod',
            'CashBalancesEndOfPeriod',
            'Depreciation',
            'PayTheInterest',	
            'PropertyAndPlantAndEquipment' ]

# 建立 StandardScaler 物件
ss = StandardScaler()

df_features_scaled = pd.DataFrame(ss.fit_transform(df[features]), columns=features, index=df.index)


df.update(df_features_scaled)

In [5]:
fe = FeatureEngineer(
                    use_technical_indicator=False,
                    tech_indicator_list = INDICATORS,
                    use_vix=False,
                    use_turbulence=False, 
                    user_defined_feature = False)

PREDICT = fe.preprocess_data(df)

In [9]:
stock_dimension = len(PREDICT.tic.unique()) 
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension 
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 74, State Space: 8585


In [10]:
buy_cost_list =  [0] * stock_dimension
sell_cost_list = [0] * stock_dimension
buy_cost_list =  [0] * stock_dimension
sell_cost_list = [0] * stock_dimension
num_stock_shares = [0] * stock_dimension


env_kwargs = {
    "hmax": 1,
    "initial_amount": 0,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 0
}

In [11]:
trained_ppo = PPO.load("PPO_Stock_Picked_Agent_Sharpe_Ratio.zip")

== CURRENT SYSTEM INFO ==
OS: Windows-10-10.0.19041-SP0 10.0.19041
Python: 3.7.13
Stable-Baselines3: 1.6.2
PyTorch: 1.13.0+cpu
GPU Enabled: False
Numpy: 1.18.5
Gym: 0.21.0

== SAVED MODEL SYSTEM INFO ==
OS: Windows-10-10.0.19041-SP0 10.0.19041
Python: 3.7.13
Stable-Baselines3: 1.6.2
PyTorch: 1.13.0+cpu
GPU Enabled: False
Numpy: 1.18.5
Gym: 0.21.0

old_net_arch:  None
new_net_arch:  [64, 64, {'pi': [64, 64, 64], 'vf': [64, 64, 64]}]


In [12]:
e_predict_gym = StockTradingEnv(df = PREDICT, turbulence_threshold = None,risk_indicator_col=None, **env_kwargs)

predict_env, predict_obs = e_predict_gym.get_sb_env()
predict_env.reset()
action, _states = trained_ppo.predict(predict_obs, deterministic=True)

print(action)

[[1 1 0 0 0 0 1 0 0 1 0 0 0 1 0 0 1 0 1 1 0 0 1 1 0 1 0 0 0 0 1 0 0 0 1 0
  1 0 1 1 0 1 1 1 1 0 0 1 0 0 1 0 0 1 0 0 0 1 1 1 0 1 1 1 0 1 1 0 1 1 0 0
  1 1]]
